## Resumen - Actividad 3

En esta sección se implementa el código de la actividad 3 de manera compacta para obtener las variables de caracterización identificadas y aplicar los filtros correspondientes.

In [105]:
# Importa las librerías necesarias
from pyspark.sql import SparkSession

# Crea una sesión de Spark
spark = SparkSession.builder.appName("YellowTripDataAnalysis").config("spark.driver.memory", "22g").getOrCreate()

# Leemos el archivo CSV de datos de viajes amarillos
df = spark.read.option("header", True).option("inferSchema", True).csv("../data/yellow_tripdata/yellow_tripdata_2015-01.csv")

In [106]:
df.describe().show()

+-------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+--------------------+------------------+-------------------+---------------------+------------------+
|summary|          VendorID|   passenger_count|     trip_distance|   pickup_longitude|   pickup_latitude|        RateCodeID|store_and_fwd_flag| dropoff_longitude|  dropoff_latitude|      payment_type|       fare_amount|              extra|             mta_tax|        tip_amount|       tolls_amount|improvement_surcharge|      total_amount|
+-------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+--------------------+------------------+-------------------+-------------------

In [107]:
# Imprimimos las dimensiones del DataFrame
original_num_rows = df.count()
original_num_cols = len(df.columns)

print(f"Número de filas originales: {original_num_rows}")
print(f"Número de columnas originales: {original_num_cols}")

Número de filas originales: 12748986
Número de columnas originales: 19


In [108]:
# Generamos un diccionario con los factores de caracterización
charact_factor = {
    "passenger_count": 1e10,
    "trip_distance": 2.5,
    "payment_type": 3,
    "total_amount": 3,
    "tpep_pickup_datetime": 1e10,
    "tpep_dropoff_datetime": 1e10,
}

# Obtenemos una muestra del 5% de los datos originales
df_original_sample = df.select(list(charact_factor.keys())).sample(False, 0.05, seed=42)

In [109]:
# Imprimimos las dimensiones del DataFrame de la muestra
original_05_num_rows = df_original_sample.count()
original_05_num_cols = len(df_original_sample.columns)

print(f"Número de filas originales: {original_num_rows}")
print(f"Número de columnas originales: {original_num_cols}")
print(f"Número de filas del 5% de los datos originales: {original_05_num_rows}")
print(f"Número de columnas del 5% de los datos originales: {original_05_num_cols}")

Número de filas originales: 12748986
Número de columnas originales: 19
Número de filas del 5% de los datos originales: 637214
Número de columnas del 5% de los datos originales: 6


In [110]:
# Importa las librerías necesarias
from pyspark.sql.functions import col, hour, when, unix_timestamp
from pyspark.sql.types import IntegerType

# Copiamos el DataFrame original a otro DataFrame
df_sample = df_original_sample.select("*")

# Se eliminan las filas con valores nulos
df_sample = df_sample.dropna()

# Se crea la columna trip_duration
df_sample = df_sample.withColumn(
    "trip_duration",
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60,
)

# Se agrega la columna trip_duration al diccionario charact_factor
charact_factor['trip_duration'] = 2

# Se extraen las horas de las columnas tpep_pickup_datetime y tpep_dropoff_datetime
df_sample = df_sample.withColumn("tpep_pickup_datetime", hour("tpep_pickup_datetime"))
df_sample = df_sample.withColumn("tpep_dropoff_datetime", hour("tpep_dropoff_datetime"))

# Se eliminan los valores negativos y ceros de las columnas trip_distance, 
# total_amount, passenger_count y trip_duration
for colname in ["trip_distance", "total_amount", "passenger_count", "trip_duration"]:
    df_sample = df_sample.filter(col(colname) > 0)

# Se crea la columna trip_distance_bind
df_sample = df_sample.withColumn(
    "trip_distance_bind",
    when(col("trip_distance") <= 3.0, 1)
    .when((col("trip_distance") > 3.0) & (col("trip_distance") <= 10.0), 2)
    .when(col("trip_distance") > 10.0, 3)
)

# Se crea la columna total_amount_bind
df_sample = df_sample.withColumn(
    "total_amount_bind",
    when(col("total_amount") <= 10.0, 1)
    .when((col("total_amount") > 10.0) & (col("total_amount") <= 20.0), 2)
    .when(col("total_amount") > 20.0, 3)
)

# Se crea la columna passenger_count_bind
df_sample = df_sample.withColumn(
    "passenger_count_bind",
    when(col("passenger_count") <= 2.0, 1)
    .when((col("passenger_count") > 2.0) & (col("passenger_count") <= 6.0), 2)
)

# Se crea la columna trip_duration_bind
df_sample = df_sample.withColumn(
    "trip_duration_bind",
    when(col("trip_duration") <= 10.0, 1)
    .when((col("trip_duration") > 10.0) & (col("trip_duration") <= 20.0), 2)
    .when(col("trip_duration") > 20.0, 3)
)

# Se crea la columna payment_type_bind
df_sample = df_sample.withColumn(
    "payment_type_bind",
    when(col("payment_type") == 1.0, 1)
    .when((col("payment_type") > 1.0) & (col("payment_type") <= 6.0), 2)
)

# Se vuelven a eliminar las filas con valores nulos
df_sample = df_sample.dropna()

# Se convierten las columnas binned a tipo Integer
for binned_col in [
    "trip_distance_bind",
    "trip_duration_bind",
    "total_amount_bind",
    "passenger_count_bind",
    "payment_type_bind",
]:
    df_sample = df_sample.withColumn(binned_col, col(binned_col).cast(IntegerType()))

# Se eliminan los outliers de las columnas numéricas
for col_name, factor in charact_factor.items():
    quantiles = df_sample.approxQuantile(col_name, [0.25, 0.75], 0.0)
    q1, q3 = quantiles
    iqr = q3 - q1
    lower = q1 - factor * iqr
    upper = q3 + factor * iqr
    df_sample = df_sample.filter((col(col_name) >= lower) & (col(col_name) <= upper))


In [111]:
# Imprimimos las dimensiones del DataFrame después de la limpieza
cleaned_05_num_rows = df_sample.count()
cleaned_05_num_cols = len(df_sample.columns)

print(f"Número de filas originales: {original_num_rows}")
print(f"Número de columnas originales: {original_num_cols}")
print(f"Número de filas del 5% de los datos originales: {original_05_num_rows}")
print(f"Número de columnas del 5% de los datos originales: {original_05_num_cols}")
print(f"Número de filas del 5%  después de la limpieza: {cleaned_05_num_rows}")
print(f"Número de columnas del 5% después de la limpieza: {cleaned_05_num_cols}")

Número de filas originales: 12748986
Número de columnas originales: 19
Número de filas del 5% de los datos originales: 637214
Número de columnas del 5% de los datos originales: 6
Número de filas del 5%  después de la limpieza: 579787
Número de columnas del 5% después de la limpieza: 12


In [112]:
df_sample.show(5)

+---------------+-------------+------------+------------+--------------------+---------------------+------------------+------------------+-----------------+--------------------+------------------+-----------------+
|passenger_count|trip_distance|payment_type|total_amount|tpep_pickup_datetime|tpep_dropoff_datetime|     trip_duration|trip_distance_bind|total_amount_bind|passenger_count_bind|trip_duration_bind|payment_type_bind|
+---------------+-------------+------------+------------+--------------------+---------------------+------------------+------------------+-----------------+--------------------+------------------+-----------------+
|              5|         2.83|           2|        14.3|                  19|                   19|15.333333333333334|                 1|                2|                   2|                 2|                2|
|              1|          3.0|           2|        12.8|                  14|                   14|13.116666666666667|                 1|  

In [113]:
# Importa las librerías necesarias
from itertools import product
from pyspark.sql.functions import col

# Iteraciones de los valores binned
iterations = {
    "total_amount_bind": [1, 2, 3],
    "trip_distance_bind": [1, 2, 3],
    "payment_type_bind": [1, 2],
    "passenger_count_bind": [1, 2]
}

# Número total de filas
total_rows = df_sample.count()

# Extracción de las claves y valores del diccionario
keys = list(iterations.keys())
values = list(iterations.values())

# Crear todas las combinaciones posibles de los valores
combinations = list(product(*values))

# Número de iteración
iter_num = 1

# Suma total de proporciones
total_proportion = 0

# Iterar sobre cada combinación
for comb in combinations:
    # Construir el filtro dinámicamente
    condition = (col(keys[0]) == comb[0])
    for i in range(1, len(keys)):
        condition &= (col(keys[i]) == comb[i])
    
    # Aplicar el filtro
    filtered_rows = df_sample.filter(condition)
    
    # Contar y calcular proporción
    count = filtered_rows.count()
    proportion = count / total_rows if total_rows != 0 else 0

    print(f"Combinación: {comb} / iteración: {iter_num}, Proporción: {proportion:.4f}")
    
    iter_num += 1
    total_proportion += proportion

# Mostrar proporción total
print(f"Proporción total: {total_proportion:.4f}")


Combinación: (1, 1, 1, 1) / iteración: 1, Proporción: 0.1930


Combinación: (1, 1, 1, 2) / iteración: 2, Proporción: 0.0327


Combinación: (1, 1, 2, 1) / iteración: 3, Proporción: 0.1954


Combinación: (1, 1, 2, 2) / iteración: 4, Proporción: 0.0362


Combinación: (1, 2, 1, 1) / iteración: 5, Proporción: 0.0000


...

Combinación: (3, 2, 2, 2) / iteración: 32, Proporción: 0.0029


Combinación: (3, 3, 1, 1) / iteración: 33, Proporción: 0.0000


Combinación: (3, 3, 1, 2) / iteración: 34, Proporción: 0.0000


Combinación: (3, 3, 2, 1) / iteración: 35, Proporción: 0.0000


Combinación: (3, 3, 2, 2) / iteración: 36, Proporción: 0.0000
Proporción total: 1.0000


In [114]:
# Establece los rangos para las columnas
total_amount_bind = [0.0, 10.0, 20.0, 100.0]
trip_distance_bind = [0.0, 3.0, 10.0, 100.0]
payment_type_bind = [0.0, 1.0, 6.0]
passenger_count_bind = [0.0, 2.0, 6.0]

# Crea listas para almacenar los rangos de cada columna
total_amount_bind_ranges = []
trip_distance_bind_ranges = []
payment_type_bind_ranges = []
passenger_count_bind_ranges = []

# Crea los rangos para cada columna
for i in range(len(total_amount_bind) - 1):
    total_amount_bind_ranges.append((total_amount_bind[i], total_amount_bind[i + 1]))
    
for i in range(len(trip_distance_bind) - 1):
    trip_distance_bind_ranges.append((trip_distance_bind[i], trip_distance_bind[i + 1]))
    
for i in range(len(payment_type_bind) - 1):
    payment_type_bind_ranges.append((payment_type_bind[i], payment_type_bind[i + 1]))
    
for i in range(len(passenger_count_bind) - 1):
    passenger_count_bind_ranges.append((passenger_count_bind[i], passenger_count_bind[i + 1]))

# Imprime los rangos de cada columna
print("Total amount bind ranges: ", total_amount_bind_ranges)
print("Trip distance bind ranges: ", trip_distance_bind_ranges)
print("Payment type bind ranges: ", payment_type_bind_ranges)
print("Passenger count bind ranges: ", passenger_count_bind_ranges)

# Crea todas las combinaciones posibles de los rangos
combinations = list(product(total_amount_bind_ranges, trip_distance_bind_ranges, payment_type_bind_ranges, passenger_count_bind_ranges))

# Imprime las combinaciones
print("Combinations: ", combinations)

Total amount bind ranges:  [(0.0, 10.0), (10.0, 20.0), (20.0, 100.0)]
Trip distance bind ranges:  [(0.0, 3.0), (3.0, 10.0), (10.0, 100.0)]
Payment type bind ranges:  [(0.0, 1.0), (1.0, 6.0)]
Passenger count bind ranges:  [(0.0, 2.0), (2.0, 6.0)]
Combinations:  [((0.0, 10.0), (0.0, 3.0), (0.0, 1.0), (0.0, 2.0)), ((0.0, 10.0), (0.0, 3.0), (0.0, 1.0), (2.0, 6.0)), ((0.0, 10.0), (0.0, 3.0), (1.0, 6.0), (0.0, 2.0)), ((0.0, 10.0), (0.0, 3.0), (1.0, 6.0), (2.0, 6.0)), ((0.0, 10.0), (3.0, 10.0), (0.0, 1.0), (0.0, 2.0)), ((0.0, 10.0), (3.0, 10.0), (0.0, 1.0), (2.0, 6.0)), ((0.0, 10.0), (3.0, 10.0), (1.0, 6.0), (0.0, 2.0)), ((0.0, 10.0), (3.0, 10.0), (1.0, 6.0), (2.0, 6.0)), ((0.0, 10.0), (10.0, 100.0), (0.0, 1.0), (0.0, 2.0)), ((0.0, 10.0), (10.0, 100.0), (0.0, 1.0), (2.0, 6.0)), ((0.0, 10.0), (10.0, 100.0), (1.0, 6.0), (0.0, 2.0)), ((0.0, 10.0), (10.0, 100.0), (1.0, 6.0), (2.0, 6.0)), ((10.0, 20.0), (0.0, 3.0), (0.0, 1.0), (0.0, 2.0)), ((10.0, 20.0), (0.0, 3.0), (0.0, 1.0), (2.0, 6.0)), ((10.0

## Actividad 4

1. Introducción teórica
2. Selección de datos
3. Preparación de los datos
4. Preparación del conjunto de entrenamiento y prueba
5. Construcción de modelos de aprendizaje supervisado y no supervisado

### 1. Introducción teórica

El aprendizaje máquina se puede divirdir claramente entre el aprendizaje supervisado y el aprendizaje no supervisado, diferenciando entre ellos que el aprendizaje supervisado utiliza etiquetas para identificar las variables dependientes y las variables independientes y entrenará el modelo para enseñarse a predecir las variables objetivo. Por el contrario, el aprendizaje no supervisado se utiliza para encontrar patrones, estos normalmente son utilizados para encontrar anomalías.

### 2. Selección de datos

Para la selección de datos se realiza el siguiente procedimiento:
- Se itera sobre cada combinación que arrojó la actividad 3, estas combinaciones contienen rangos válidos de las variables de caracterización calculadas en la actividad 3.
- Se calcula la proporcion de datos para la combinación actual.
- Si la proporcion es mayor al 5% de datos, agregamos el 40% de ese dataframe filtrado (con el propósito de limitar la cantidad de datos).

In [115]:
# Importando librerías necesarias
from pyspark.sql.types import StructType, StructField, IntegerType

# Conteo de filas del DataFrame original
total_rows = df.count()

# Variables para almacenar los resultados
subsets = []
iter_num = 1
total_proportion = 0
total_proportion_in_new_df = 0
df_accumulated = None

# Iterar sobre cada combinación
for comb in combinations:
    filtered_rows = df.filter(
        (col("total_amount") > comb[0][0]) & (col("total_amount") <= comb[0][1]) &
        (col("trip_distance") > comb[1][0]) & (col("trip_distance") <= comb[1][1]) &
        (col("payment_type") > comb[2][0]) & (col("payment_type") <= comb[2][1]) &
        (col("passenger_count") > comb[3][0]) & (col("passenger_count") <= comb[3][1])
    )
    subsets.append(filtered_rows)
    
    # Calcular la proporción de filas filtradas respecto al total
    prop = filtered_rows.count() / total_rows
    total_proportion += prop
    print(f"Porcentaje: {total_proportion:.2f} / combinación: {comb} / iteración: {iter_num} / proporción: {prop:.4f}")
    iter_num += 1

    # Si la proporción es mayor que 0.05, tomar el 40% de las filas
    if prop > 0.05:

        # Tomar el 40% de las filas filtradas
        filtered_rows_fraction = filtered_rows.sample(withReplacement=False, fraction=0.4, seed=42)

        if df_accumulated is None:
            df_accumulated = filtered_rows_fraction
        else:
            df_accumulated = df_accumulated.union(filtered_rows_fraction)

        # Guarda la proporción de filas filtradas respecto al total
        total_proportion_in_new_df += filtered_rows.count() / total_rows

        print("Combination accepted")
    
# Imprime resultados
print(f"Proporción total: {total_proportion:.4f}")
print(f"Proporción total en el nuevo DataFrame: {total_proportion_in_new_df:.4f}")

Porcentaje: 0.18 / combinación: ((0.0, 10.0), (0.0, 3.0), (0.0, 1.0), (0.0, 2.0)) / iteración: 1 / proporción: 0.1753


Combination accepted


Porcentaje: 0.21 / combinación: ((0.0, 10.0), (0.0, 3.0), (0.0, 1.0), (2.0, 6.0)) / iteración: 2 / proporción: 0.0298


Porcentaje: 0.38 / combinación: ((0.0, 10.0), (0.0, 3.0), (1.0, 6.0), (0.0, 2.0)) / iteración: 3 / proporción: 0.1786


Combination accepted


...

Porcentaje: 0.95 / combinación: ((20.0, 100.0), (3.0, 10.0), (1.0, 6.0), (2.0, 6.0)) / iteración: 32 / proporción: 0.0042


Porcentaje: 0.97 / combinación: ((20.0, 100.0), (10.0, 100.0), (0.0, 1.0), (0.0, 2.0)) / iteración: 33 / proporción: 0.0259


Porcentaje: 0.98 / combinación: ((20.0, 100.0), (10.0, 100.0), (0.0, 1.0), (2.0, 6.0)) / iteración: 34 / proporción: 0.0046


Porcentaje: 0.99 / combinación: ((20.0, 100.0), (10.0, 100.0), (1.0, 6.0), (0.0, 2.0)) / iteración: 35 / proporción: 0.0121


Porcentaje: 0.99 / combinación: ((20.0, 100.0), (10.0, 100.0), (1.0, 6.0), (2.0, 6.0)) / iteración: 36 / proporción: 0.0026
Proporción total: 0.9922
Proporción total en el nuevo DataFrame: 0.7011


In [116]:
# Imprime distintas dimensiones pasadas
print(f"Número de filas originales: {original_num_rows}")
print(f"Número de columnas originales: {original_num_cols}")
print(f"Número de filas del 5% de los datos originales: {original_05_num_rows}")
print(f"Número de columnas del 5% de los datos originales: {original_05_num_cols}")
print(f"Número de filas del 5%  después de la limpieza: {cleaned_05_num_rows}")
print(f"Número de columnas del 5% después de la limpieza: {cleaned_05_num_cols}")

# Imprime dimensiones del DataFrame acumulado
num_rows_after_dimension_extraction = df_accumulated.count()
num_cols_after_dimension_extraction = len(df_accumulated.columns)
print(f"Número de filas despues de la extración contenida de dimensiones: {num_rows_after_dimension_extraction}")
print(f"Número de columnas despues de la extración contenida de dimensiones: {num_cols_after_dimension_extraction}")

Número de filas originales: 12748986
Número de columnas originales: 19
Número de filas del 5% de los datos originales: 637214
Número de columnas del 5% de los datos originales: 6
Número de filas del 5%  después de la limpieza: 579787
Número de columnas del 5% después de la limpieza: 12


Número de filas despues de la extración contenida de dimensiones: 3582756
Número de columnas despues de la extración contenida de dimensiones: 19


### 3. Preparación de los datos

Se realiza una limpieza de los datos de la siguiente manera:
- Se crea una copia del dataframe donde se encuentra la muestra contenida.
- Se eliminan las columnas innecesarias.
- Se eliminan las columnas con valores nulos.
- Se crea la columna trip_duration para tener el dato de la duración de los viajes en minutos.
- Se dejan solo las horas del viaje en las columnas tpep_pickup_datetime y tpep_dropoff_datetime.
- Se eliminan los valores negatios y ceros de las columnas. De la columna tip_amount solo se eliminan los valores negativos.
- Se filtran los outliers. Solo aplica para tip_amount y trip_duration, ya que el resto de datos fue filtrado por los filtros de la creación de la muestra contenida.

In [117]:
# Preprocesamiento de datos

# Se crea una copia del DataFrame original
df_filtered = df_accumulated.select(['*'])

# Listado de columnas innecesarias
columns_to_drop = [
    "VendorID",
    "pickup_longitude",
    "pickup_latitude",
    "RateCodeID",
    "store_and_fwd_flag",
    "dropoff_longitude",
    "dropoff_latitude",
    "fare_amount",
    "extra",
    "mta_tax",
    "tolls_amount",
    "improvement_surcharge",
]

# Se eliminan las columnas innecesarias
for colname in columns_to_drop:
    df_filtered = df_filtered.drop(colname)

# Se eliminan las filas con valores nulos
df_filtered = df_filtered.dropna()

# Se crea la columna trip_duration
df_filtered = df_filtered.withColumn(
    "trip_duration",
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60,
)

# Se extraen las horas de las columnas tpep_pickup_datetime y tpep_dropoff_datetime
df_filtered = df_filtered.withColumn("tpep_pickup_datetime", hour("tpep_pickup_datetime"))
df_filtered = df_filtered.withColumn("tpep_dropoff_datetime", hour("tpep_dropoff_datetime"))

# Se eliminan los valores negativos y ceros de las columnas passenger_count,
# trip_distance, payment_type, tip_amount, total_amount y trip_duration
for colname in ["passenger_count", "trip_distance", "payment_type", "tip_amount", "total_amount", "trip_duration"]:

    # Condición: si la columna es "tip_amount", se permite cero por que no
    # todos los viajes tienen propina
    if colname != "tip_amount":
        df_filtered = df_filtered.filter(col(colname) > 0)
    else:
        df_filtered = df_filtered.filter(col(colname) >= 0)

# Se filtran los outliers de las columnas numéricas
# Solo se consideran las columnas que no fueron segmentadas en el filtrado
outlier_columns = [
    "tip_amount",
    "trip_duration"
    ]

# Se eliminan los outliers de las columnas numéricas
for col_name in outlier_columns:
    quantiles = df_filtered.approxQuantile(col_name, [0.25, 0.75], 0.0)
    q1, q3 = quantiles
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    df_filtered = df_filtered.filter((col(col_name) >= lower) & (col(col_name) <= upper))

In [118]:
num_rows_after_dimension_extraction_cleaned = df_filtered.count()
num_cols_after_dimension_extraction_cleaned = len(df_filtered.columns)

print(f"Número de filas despues de la extración contenida de dimensiones: {num_rows_after_dimension_extraction}")
print(f"Número de columnas despues de la extración contenida de dimensiones: {num_cols_after_dimension_extraction}")
print(f"Número de filas despues de la limpieza de la extración contenida de dimensiones: {num_rows_after_dimension_extraction_cleaned}")
print(f"Número de columnas despues de la limpieza de la extración contenida de dimensiones: {num_cols_after_dimension_extraction_cleaned}")

Número de filas despues de la extración contenida de dimensiones: 3582756
Número de columnas despues de la extración contenida de dimensiones: 19
Número de filas despues de la limpieza de la extración contenida de dimensiones: 3352867
Número de columnas despues de la limpieza de la extración contenida de dimensiones: 8


### 4. Preparación del conjunto de entrenamiento y prueba

- Se utiliza una separación del conjunto de 85/15, ya que tenemos más de 3 millones de datos, tener aproximadamente 500,000 datos para validación es suficiente.
- Se utiliza una semilla fija para tener reproducibilidad.


In [119]:
# Separamos los datos en conjuntos de entrenamiento y prueba
train_df, test_df = df_filtered.randomSplit([0.85, 0.15], seed=42)

# Tamaño del conjunto de entrenamiento
train_size = train_df.count()
print(f"Tamaño del conjunto de entrenamiento: {train_size}")

# Tamaño del conjunto de prueba
test_size = test_df.count()
print(f"Tamaño del conjunto de prueba: {test_size}")

Tamaño del conjunto de entrenamiento: 2849002


Tamaño del conjunto de prueba: 503865


### 5. Construcción de modelos de aprendizaje supervisado y no supervisado

- Para entrenar el modelo, se le otorga a PySpark una RAM de 22 GB (esto se realiza en la creación la sesión de spark). Esto a consecuencia de que el modelo se quedaba sin memoria para procesar todos los datos.

In [120]:
# Copiamos el DataFrame de entrenamiento y prueba a otros DataFrames

# Copias para el entrenamiento y prueba del modelo supervisado
supervised_train_df = train_df.select(['*'])
supervised_test_df = test_df.select(['*'])

# Copias para el entrenamiento y prueba del modelo no supervisado
unsupervised_train_df = train_df.select(['*'])
unsupervised_test_df = test_df.select(['*'])

# Se muestra la RAM asignada al driver
memory_used = spark.sparkContext.getConf().get("spark.driver.memory")
print(f"Driver memory: {memory_used}")

Driver memory: 22g


#### 5.1 Modelo de aprendizaje supervisado

- Se elige RandomForestRegresor.

Pasos en el código:

- Se definen las columnas numéricas que se utilizarán como las caracterísitcas (variables independientes).
- Se ensamblan las caracterísitcas en un solo vector con VectorAssembler, esto se realiza así por que PySpark ML no acepta múltiples columnas si no una sola columna con todas las variables independientes empaquetadas en un vector.
- Se define el modelo, en este caso se utiliza RandomForestRegressor.
- Se entrena el modelo y se carga a la variable rf_model.
- Se realizan las predicciones.
- Se evalúa el modelo.
- Se obtiene un RMSE de 1.2659, este es un valor sumamente aceptable, ya que indica que nuestro error es de $1.3 USD respecto al total_amount que es nuestra variable dependiente.
- Creamos un DataFrame nuevo con datos manuales para verificar el comportamiento del modelo, nos basamos en los primeros 5 datos del DataFrame original, cambiando ligeros aspectos, lo que nos debe dar como resultado un total_amount bastante parecido.
- Ensamblamos el nuevo DataFrame en un solo vector, se hacen las predicciones e imprimimos resultados, después de analizar los resultados, se observa que obtenemos un total_amount bastante parecido, lo que indica que el modelo opera con suma precisión.


In [121]:
# Importamos las librerías necesarias
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# Se definen las columnas numéricas que se usarán como features
# No se incluye tip_amount ni total_amount
numeric_cols = [
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "passenger_count",
    "trip_distance",
    "payment_type",
    "trip_duration",
]

# Se ensamblan las features en un solo vector
assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")
train_df_vec = assembler.transform(supervised_train_df)
test_df_vec = assembler.transform(supervised_test_df)

# Se define el modelo
rf = RandomForestRegressor(
    featuresCol="features",
    labelCol="total_amount",
    numTrees=80,
)

# Se entrena el modelo
rf_model = rf.fit(train_df_vec)

# Se hacen predicciones
predictions = rf_model.transform(test_df_vec)

# Se evalúa el modelo
evaluator = RegressionEvaluator(
    labelCol="total_amount",
    predictionCol="prediction",
    metricName="rmse",
)
rmse = evaluator.evaluate(predictions)

# Se imprime el RMSE
print(f"RMSE: {rmse:.4f}")

RMSE: 1.2659


In [122]:
# Importamos las librerías necesarias
from pyspark.sql.types import DoubleType

# Creamos un nuevo DataFrame para probar el modelo

test_manual_data = [
    (19, 19, 5, 2.9, 2, 14.5),
    (14, 14, 1, 3.2, 2, 12.9),
    (14, 14, 2, 1.4, 1, 16.5),
    (12, 12, 1, 3.9, 1, 18.2),
    (12, 12, 1, 0.8, 1, 6.5),
]

# Definimos el esquema del DataFrame, trip_distance is double type
schema = StructType([
    StructField("tpep_pickup_datetime", IntegerType(), True),
    StructField("tpep_dropoff_datetime", IntegerType(), True),
    StructField("passenger_count", IntegerType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("payment_type", IntegerType(), True),
    StructField("trip_duration", DoubleType(), True),
])


# Creamos el DataFrame
test_manual_df = spark.createDataFrame(test_manual_data, schema)

# Mostramos el DataFrame con los datos de prueba manuales
test_manual_df.show()

+--------------------+---------------------+---------------+-------------+------------+-------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|payment_type|trip_duration|
+--------------------+---------------------+---------------+-------------+------------+-------------+
|                  19|                   19|              5|          2.9|           2|         14.5|
|                  14|                   14|              1|          3.2|           2|         12.9|
|                  14|                   14|              2|          1.4|           1|         16.5|
|                  12|                   12|              1|          3.9|           1|         18.2|
|                  12|                   12|              1|          0.8|           1|          6.5|
+--------------------+---------------------+---------------+-------------+------------+-------------+



In [123]:
# Se ensambla el nuevo DataFrame en un solo vector
test_manual_df_vec = assembler.transform(test_manual_df)

# Se hacen predicciones
manual_predictions = rf_model.transform(test_manual_df_vec)

# Se muestran las características y las predicciones
manual_predictions.select("features", "prediction").show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[19.0,19.0,5.0,2....|13.190304452326487|
|[14.0,14.0,1.0,3....|   12.166617641157|
|[14.0,14.0,2.0,1....|13.732458063128016|
|[12.0,12.0,1.0,3....|22.103485518209602|
|[12.0,12.0,1.0,0....| 8.214127753036887|
+--------------------+------------------+



#### 5.1 Modelo no supervisado

- Se elige KMeans.

Pasos en el código:

- Se definen las columnas numéricas.
- Se combina el DataFrame de entrenamiento y prueba.
- Se ensamblan las columnas en un solo vector con VectorAssembler.
- Se definel el modelo KMeans.
- Se entrena el modelo KMeans con el DataFrame combinado.
- Se hacen las predicciones.
- Se evalúa el modelo.
- Obtenemos un índice Silhouette de 0.5353 (distancia entre puntos en su propio cluster). Un valor de 0.5363 es aceptable, aun que puede mejorar con algunas otras técnicas.

In [124]:
# Importamos las librerías necesarias
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Definimos las columnas numéricas
numeric_cols = [
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "passenger_count",
    "trip_distance",
    "payment_type",
    "trip_duration",
]

# Combinamos el DataFrame de entrenamiento y prueba
unsupervised_complete_df = unsupervised_train_df.union(unsupervised_test_df)

# Se ensamblan las features en un solo vector
assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")
train_df_vec = assembler.transform(unsupervised_complete_df)

# Se define el modelo
kmeans = KMeans(k=4, seed=42)

# Se entrena el modelo
km_model = kmeans.fit(train_df_vec)

# Se hacen predicciones
predictions = km_model.transform(train_df_vec)

# Se evalúa el modelo
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)

# Se imprime el Silhouette
print(f"Silhouette with squared euclidean distance = {silhouette:.4f}")

Silhouette with squared euclidean distance = 0.5363
